In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
sys.path.append("..")

# Emergency Department Notes
## Overview
When a patient visits the emergency department (ED), the physician may need to make a quick diagnostic decision with little information. If the provider suspects pneumonia, they will document this and other possible diagnoses in the note. By the end of the encounter, they may have a clear diagnosis or may still be uncertain.

The goal of the emergency pipeline is to capture suspected or definitive diagnostic assertions of pneumonia in emergency notes.

While pneumonia may be mentioned throughout the note, we need to limit the classification logic to look at certain sections of the note which are most meaningful for a diagnosis. Additionally, some mentions of pneumonia may be negated and should indicate a negative diagnosis.

The diagram below walks through the logic used for classifying emergency notes:
![Emergency classification](./Emergency.png)

In words:
- First, the system looks for ***Tier 1*** sections such as **"Assessment/Plan"** and **"Diagnoses"** sections. These represent the final diagnosis of the physician. If either of these contain a positive or negative mention of pneumonia, the NLP makes the appropriate classification.
- If those sections aren't in the note or don't contain pneumonia, the NLP looks for ***Tier 2*** sections such as **"Medical Decision Making"** or **"ED Course"** sections. These will document possible diagnoses that the provider is considering. If no mentions of pneumonia are found, then the document is classified as **"Negative"**.
- However, if a positive mention is found, it may only have been documented as a potential diagnosis that the provider considered but ultimately ruled out. To account for this, the NLP next looks for mentions of negative radiographic findings in any of the sections mentioned already. For example, if the MDM says that pneumonia is a possible diagnosis but later says that there was a negative X-Ray and pneumonia isn't mentioned anywhere else in the note, this will be considered **"Negative"**. If there are no such negative findings, the document is classified as **"Possible"/"Positive"**

The rest of this notebook will walk through some specific examples.

In [3]:
from medspacy_pna import build_nlp
from medspacy_pna.display import create_html
from medspacy.visualization import visualize_ent
from IPython.display import HTML

In [4]:
%%capture
nlp_emergency = build_nlp("emergency")

In [5]:
clf = nlp_emergency.get_pipe("pneumonia_emergencydocumentclassifier")

In [6]:
# Sections used for classification
from medspacy_pna.document_classification.emergency_document_classifier import RELEVANT_SECTIONS
RELEVANT_SECTIONS

{'TIER_1': {'addendum',
  'diagnoses',
  'discharge_diagnoses',
  'impression',
  'observation_and_plan'},
 'TIER_2': {'admission_diagnoses',
  'ed_course',
  'hospital_course',
  'medical_decision_making'}}

In [7]:
nlp_emergency.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'medspacy_concept_tagger',
 'medspacy_target_matcher',
 'medspacy_context',
 'medspacy_sectionizer',
 'medspacy_postprocessor',
 'pneumonia_emergencydocumentclassifier']

## Example 1: Positive in Asssessment/Plan and Diagnoses

In [8]:
text = """
Hospital Course: The patient presents with a cough and fever.
A chest x-ray was positive for opacities. 

Final Diagnosis: Pneumonia.
"""
doc = nlp_emergency(text)

In [9]:
HTML(create_html(doc, "emergency", document_classification=True))

In [10]:
for ent in doc.ents:
    print(ent, ent.label_, ent._.section_category)

opacities OPACITY hospital_course
Pneumonia PNEUMONIA discharge_diagnoses


In [11]:
print(doc._.document_classification)

POS


## Example 2: Negative pneumonia

In [12]:
text = """
Hospital Course: The patient presents with a cough and fever.
A chest x-ray was negative for opacities. 

Assessment/Plan: No pneumonia. Admit for further monitoring.
"""
doc = nlp_emergency(text)

In [13]:
HTML(create_html(doc, "emergency", document_classification=True))

In [14]:
for ent in doc.ents:
    print(ent, ent.label_, ent._.section_category)

opacities OPACITY hospital_course
pneumonia PNEUMONIA observation_and_plan


In [15]:
print(doc._.document_classification)

NEG


## Example 3: Possible in MDM

In [16]:
text = """
Hospital Course: The patient presents with a cough and fever.
A chest x-ray was positive for opacities. 

Medical decision making: Possible differential dx include pneumonia, atelectasis, and CHF.

"""
doc = nlp_emergency(text)

In [17]:
HTML(create_html(doc, "emergency", document_classification=True))

In [18]:
for ent in doc.ents:
    print(ent, ent.label_, ent._.section_category)

opacities OPACITY hospital_course
pneumonia PNEUMONIA medical_decision_making
atelectasis ATELECTASIS medical_decision_making


In [19]:
print(doc._.document_classification)

POSSIBLE


## Example 4: Negated in MDM
In this example, pneumonia is initially considered as a possible diagnosis but is later ruled out by a chest x-ray.

In [20]:
text = """
Medical decision making:
Initial presentation was concerning for pneumonia. However, CXR showed no infiltrate.

"""
doc = nlp_emergency(text)

In [21]:
HTML(create_html(doc, "emergency", document_classification=True))

In [22]:
for ent in doc.ents:
    print(ent, ent.label_, ent._.section_category, ent._.is_negated)

pneumonia PNEUMONIA medical_decision_making False
infiltrate INFILTRATE medical_decision_making True
